In [97]:
using Symbolics
using LinearAlgebra
using SHA

# generate sparse polynomials

In [2]:
@variables x[1:36]

1-element Vector{Symbolics.Arr{Num, 1}}:
 x[1:36]

In [4]:
function modify_coeffcients(A,q)
    
    coeffs_map=Symbolics.value(A).dict#turn the polynomial into matrix
    B=0#create our new polynomial
    for (key, value) in coeffs_map#iterate through the dict(monomial, coeffcients)
        if value>q
            new_coef=value%q
            B+=new_coef*key
        else
            B+=key*value
        end  
    end
    B
end

modify_coeffcients (generic function with 1 method)

In [10]:
#t monomials
#bound the degree of mononials by b, 1-b inclusive
#coefficients go from 1 to q-1(inclusive)
function sparse_polynomial(n,t,b,q)
    @variables x[1:n]
    
    sparse_poly=0
    
    for i in 1:t #we want t monomials
        deg=rand(1:b)
        
        monomial_d=1
        for j in 1:deg
            monomial_d*=x[rand(1:n)]
        end
        sparse_poly+=rand(1:q)*monomial_d
        
    end
#     println(sparse_poly)
    A=modify_coeffcients(sparse_poly,q)
    return A
end

A=sparse_polynomial(4,5,3,6)
A

2(x[3]^2)*x[1] + 2(x[1]^2)*x[3] + 2(x[2]^2)*x[3] + 6x[2]*x[4] + x[1]*x[2]*x[4]

# Generate kl matrix

In [8]:
function num_identity(k)
    a=zeros(Num,k)
    a[1]=1
    for i in 2:k
        b=zeros(Num,k)
        b[i]=1
        a=[a b] 
    end
    return a
end
I_mat=num_identity(4)
I_mat[1,1]=x[2]*x[3]+x[1]
I_mat

4×4 Matrix{Num}:
 x[2]*x[3] + x[1]  0  0  0
                0  1  0  0
                0  0  1  0
                0  0  0  1

In [11]:
function generate_U(k,n,t,B,q)
    I_matrix=num_identity(k)#generate a k by k identity matrix
    for i in 1:Int((k*k-k)/2)
        b=rand(2:k)
        a=rand(1:(b-1))
        I_matrix[a,b]=sparse_polynomial(n,t,B,q)
    end

    return I_matrix
end 
U=generate_U(4,32,4,3,6)
U

4×4 Matrix{Num}:
 1  …  4x[7] + 3x[20] + 5x[14] + 3x[6]*x[25]*x[32]
 0      4x[27] + 2x[31] + 2x[4]*x[27] + 5x[5]*x[7]
 0                                               0
 0                                               1

In [13]:
# simplify.(expand.(inv(U)))
# inv(U)

4×4 Matrix{Num}:
 1.0  …  2x[10]*x[31] + 4x[10]*x[27] + 2(x[4]^2)*x[21]*x[27] + 5x[5]*x[7]*x[10] + 2x[4]*x[10]*x[27] + 2x[4]*x[21]*x[31] + 4x[4]*x[21]*x[27] + 12x[19]*x[27]*x[29] + 6x[19]*x[29]*x[31] + 10x[1]*x[19]*x[26]*x[31] + 20x[1]*x[19]*x[26]*x[27] + 5x[4]*x[5]*x[7]*x[21] + 6x[4]*x[19]*x[27]*x[29] + 15x[5]*x[7]*x[19]*x[29] + 25x[1]*x[5]*x[7]*x[19]*x[26] + 10x[1]*x[4]*x[19]*x[26]*x[27] - 4x[7] - 5x[14] - 3x[20] - 3x[6]*x[25]*x[32]
 0                                                                                                                                                                                                                                                                                                                                                                                        -4x[27] - 2x[31] - 2x[4]*x[27] - 5x[5]*x[7]
 0                                                                                                                                         

In [15]:
function generate_L(k,n,t,B,q)
    I_matrix=num_identity(k)#generate a k by k identity matrix
    for i in 1:Int((k*k-k)/2)
        a=rand(2:k)
        b=rand(1:a-1)
        I_matrix[a,b]=sparse_polynomial(n,t,B,q)
    end
    return I_matrix
end

generate_L (generic function with 1 method)

In [16]:
function cols_to_keep(k,l)
    col_opts=[i for i in 1:k]
    for i in 1:(k-l)
        random_number=size(col_opts,1)
        splice!(col_opts, rand(1:random_number))
    end
    return col_opts
end
cols_to_keep(4,3)

3-element Vector{Int64}:
 1
 2
 3

In [17]:
function random_del_cols(k,l,matrix)
#     col_opts=[i for i in 1:l]
    col_opts=cols_to_keep(k,l)
    matrix_kl=zeros(Num, k,l)
    
    j=1
    for i in col_opts
        matrix_kl[:,j]=matrix[:,i]
        j+=1
    end
    return col_opts,matrix_kl
end

random_del_cols (generic function with 1 method)

In [18]:
function find_kl_inverse(to_keep,S_inv)
    k=size(S_inv,1)#find the number of rows
    ml_inv=zeros(Num, length(to_keep),k)
    
    j=1
    for i in to_keep
        ml_inv[j,:]=S_inv[i,:]
        j+=1
    end
    return ml_inv
end

find_kl_inverse (generic function with 1 method)

In [23]:
function modify_neg_coeff(A,q)
    
    coeffs_map=Symbolics.value(A).dict#turn the polynomial into matrix
    B=0#create our new polynomial
    for (key, value) in coeffs_map#iterate through the dict(monomial, coeffcients)
        if value>q || value<-q 
            new_coef=mod(value,q)
            B+=new_coef*key
        else
            B+=key*value
        end  
    end
    B
end

function modify_matrix_coef(matrix,q)
    res=simplify.(expand.(matrix))#expand and then simplify
    
    for i in 1:size(res,1)
        for j in 1:size(res,2)
            a=res[i,j]

            if isone(a)==false && iszero(a)==false            
                res[i,j]=modify_neg_coeff(a,q)
            end
        end
    end
    return res
end

modify_matrix_coef (generic function with 1 method)

In [46]:
function random_permutation(k)
    cols=[i for i in 1:k]
    ncols=[]
    for i in 1:k
        random_number=size(cols,1)
        a=rand(1:random_number)
        append!(ncols,cols[a])
        splice!(cols, a)
    end
    A=zeros(Num,k,k)
    
    for i in 1:k
        A[i,ncols[i]]=1
    end
    return A
end
    
P=random_permutation(4)
inv(P)*P


4×4 Matrix{Num}:
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0

In [94]:
function matrix_kl(k,l,s,n,t,B,q)
    Ls=[]
    Us=[]
    for i in 1:s
        U=generate_U(k,n,t,B,q)
        L=generate_L(k,n,t,B,q)
        push!(Us,U)
        push!(Ls,L)
    end 
    U_new=Us[rand(1:s)]
    L_new=Ls[rand(1:s)]
    
    P1=random_permutation(k)
    P2=random_permutation(k)
    kk_matrix=U_new*P1*L_new*P2
    kk_matrix=simplify.(expand.(kk_matrix))
#     kk_matrix=modify_matrix_coef(kk_matrix,q)#modify the coefficients for kk_matrix
    kk_inv=inv(P2)*inv(L_new)*inv(P1)*inv(U_new)#able to compute, not able to verify the result
    
    col_keep,kl_matrix=random_del_cols(k,l,kk_matrix)
    kl_inverse=find_kl_inverse(col_keep,kk_inv)
  
    return kk_matrix,kk_inv,kl_matrix, kl_inverse
end 

matrix_kl (generic function with 1 method)

In [117]:
A,B,C,D=matrix_kl(10,5,5,32,3,5,6)
println(size(C))
print(size(D))

(10, 5)
(5, 10)

In [118]:
res=simplify.(expand.(D*C))

5×5 Matrix{Num}:
 1.0  0    0.0  0.0  0
 0.0  1.0  0.0  0.0  0.0
 0    0    1.0  0    0
 0    0    0.0  1.0  0
 0.0  0.0  0.0  0.0  1.0

# hashing 

In [107]:
message = "Hello, world!"
@variables x[1:64]
function hashing512(message)
    hash = sha2_512(message)
    bitstring512=""
    for i in hash
        bitstring512=bitstring512*bitstring(i)
    end
    
    A512=[]
    for i in bitstring512
        append!(A512,Int(i)-48)
    end
    return A512
end


hashing512 (generic function with 1 method)

In [123]:
function polys(A512)
    coeffs=[]
    last12=A512[501:512,]
    for i in 0:3
        b3=last12[(1+3*i):3*(i+1),]
        coef=0
        for j in 1:3
            coef=coef+2^(3-j)*b3[j]
        end
        coef=mod(coef,6)
        append!(coeffs,coef)
    end
#     println(coeffs)
    
    first300=A512[1:300]
    vars=[]
    for i in 1:50
        v=first300[(1+6*(i-1)):(6*i),]
        vn=0
        for j in 1:6
            vn=vn+2^(6-j)*v[j]
        end
        vn=vn+1
        append!(vars,vn)
    end
#     println(vars)
    
    middle200=A512[301:500]
    mons=[]
    for i in 1:5
        b40=middle200[(1+40*(i-1)):(40*i)]

        for j in 1:4
            b10=b40[(1+10*(j-1)):(10*j)]
            mon=1

            for k in 1:10
                if b10[k]==1 

                    if vars[10*(j-1)+k]!=0
                        mon=mon*x[vars[10*(j-1)+k]]
                    end
                end
            end
            append!(mons,mon)

        end
    end
#     println(mons)
    
    polys=[]
    for i in 1:5
        pol=0
        for j in 1:4
            pol=pol+mons[4*(i-1)+j]*coeffs[j]
        end
        append!(polys,pol)
    end
    Un=zeros(Num,1,5)
    for i in 1:5
        Un[1,i]=U[i]
    end
    return Un
end

U=polys(A512)

1×5 Matrix{Num}:
 4x[62] + 2(x[10]^3)*x[2]*x[8]*x[16]*x[49]*x[61] + x[6]*x[11]*x[13]*x[21]*x[25]*x[44]*x[49]*x[55]  …  x[6]*x[10]*x[11]*x[13]*x[25]*x[33]*x[44] + 2(x[10]^3)*x[8]*x[16]*x[22]*x[49]*x[61] + 4x[19]*x[24]*x[30]*x[46]*x[47]*x[51]*x[62]

In [130]:
V=Un*D
V

1×10 Matrix{Num}:
 x[11]*x[13]*x[25]*x[44] + 2x[2]*x[8]*x[10]*x[55] + 4x[19]*x[30]*x[34]*x[46]*x[51]*x[54]  …  (1.0 - (6x[5]*x[15] + 3x[9]*x[12]*x[22] + 6x[1]*x[6]*x[8]*x[15]*x[22])*(-6x[13]*x[18]*x[31] - 5x[9]*x[11]*x[14]*x[21] - 3x[1]*x[3]*x[9]*x[11]*x[27])*(-(-6x[16] - x[11]*x[20] - x[16]*x[22]*x[25]*x[28])*(5x[8]*x[13] + 3x[3]*x[16] + 3x[18]*x[20]*x[22]*x[29]) - (6x[3]*x[21]*x[30] + 6x[6]*x[12]*x[21]*x[25] + x[29])*(-3x[11] - (-6x[16] - x[11]*x[20] - x[16]*x[22]*x[25]*x[28])*(4x[24] + 3x[5]*x[32] + 2x[1]*x[6]*x[19]*x[22]*x[27]) - 6x[17]*x[19]*x[27] - 5x[9]*x[16]*x[25]*x[27]) - 4x[2]*x[26] - 3x[7]*x[31]*x[32] - 5x[1]*x[10]*x[17]*x[20]*x[31]))*(4x[62] + 2(x[10]^3)*x[2]*x[8]*x[16]*x[49]*x[61] + x[6]*x[11]*x[13]*x[21]*x[25]*x[44]*x[49]*x[55]) + (2(x[10]^2)*x[49] + x[10]*x[11]*x[13]*x[33]*x[44] + 4x[19]*x[30]*x[34]*x[47]*x[51]*x[54])*((-2x[6] - 3(x[2]^2)*x[15] - 2x[5]*x[14]*x[25]*x[26])*(-3x[11] - (-6x[16] - x[11]*x[20] - x[16]*x[22]*x[25]*x[28])*(4x[24] + 3x[5]*x[32] + 2x[1]*x[6]*x[19]

In [132]:
V[3]

(-6x[13]*x[18]*x[31] - 5x[9]*x[11]*x[14]*x[21] - 3x[1]*x[3]*x[9]*x[11]*x[27])*(4x[62] + 2(x[10]^3)*x[2]*x[8]*x[16]*x[49]*x[61] + x[6]*x[11]*x[13]*x[21]*x[25]*x[44]*x[49]*x[55]) + (2(x[10]^2)*x[49] + x[10]*x[11]*x[13]*x[33]*x[44] + 4x[19]*x[30]*x[34]*x[47]*x[51]*x[54])*(-x[10] - (2x[6] + 3(x[2]^2)*x[15] + 2x[5]*x[14]*x[25]*x[26])*(-(3x[7] + 2x[2]*x[3] + 2x[6]*x[9]*x[18]*x[26]*x[28])*(-(x[15]*x[30] + 3(x[23]^2)*x[5]*x[14] + 6x[12]*x[15]*x[20]*x[21])*(-6x[13]*x[18]*x[31] - 5x[9]*x[11]*x[14]*x[21] - 3x[1]*x[3]*x[9]*x[11]*x[27]) - 5(x[11]^2)*x[16]*x[19] - 6(x[3]^2)*x[21]*x[23]*x[26] - 4x[14]*x[20]*x[22]*x[23]) - 2x[5]*x[16] - 5x[10]*x[14]*x[22] - 4x[16]*x[19]*x[21]*x[27]*x[31]) - (6x[19] + x[5]*x[20] + x[11])*(-6x[13]*x[18]*x[31] - 5x[9]*x[11]*x[14]*x[21] - 3x[1]*x[3]*x[9]*x[11]*x[27]) - 4x[16]*x[28] - 5x[9]*x[18]*x[19]*x[29]) + (x[11]*x[13]*x[25]*x[44] + 2x[2]*x[8]*x[10]*x[55] + 4x[19]*x[30]*x[34]*x[46]*x[51]*x[54])*(-2x[24] - 6x[27] - (x[2]*x[19] + 3x[29] + 3x[19]*x[24]*x[25])*(-x[10] - 4